In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd



In [ ]:
path="/content/drive/MyDrive/daa dataset/Spotify interactions.csv"
df=pd.read_csv(path)


In [ ]:

import pandas as pd
from collections import defaultdict, deque
import csv

path = "/content/drive/MyDrive/daa dataset/Spotify interactions.csv"
df = pd.read_csv(path)

print("DataFrame Head:")
print(df.head())
print("\nUnique Users:")
print(df['User Name'].unique())
print("\nUnique Songs:")
print(df['Song Title'].unique())

# Building adjacency list
def build_graph(df):
    graph = defaultdict(list)
    for index, row in df.iterrows():
        user = row['User Name']
        song = row['Song Title']
        # undirected graph
        graph[user].append(song)
        graph[song].append(user)
    return graph

#  BFS to find recommendations and hops
def bfs_recommendations(graph, start_user):
    visited = set()
    queue = deque([(start_user, 0)])
    recommendations = defaultdict(list)

    while queue:
        node, hops = queue.popleft()
        if node not in visited:
            visited.add(node)

            # current node is a song
            if node != start_user and node in df['Song Title'].values:
                recommendations[hops].append(node)  # Group songs by hop count

            #  neighbors with inc hop count
            for neighbor in graph[node]:
                if neighbor not in visited:
                    queue.append((neighbor, hops + 1))

    #  dictionary to a list of tuples
    recommendations_list = [(start_user, hops, ", ".join(songs)) for hops, songs in recommendations.items()]
    return recommendations_list

# Save to a CSV file
def save_recommendations_to_csv(all_recommendations, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['User Name', 'Hops', 'Recommended Songs'])
        for recommendation in all_recommendations:
            writer.writerows(recommendation)

# Main function
if __name__ == "__main__":

    graph = build_graph(df)
    unique_users = df['User Name'].unique()
    all_recommendations = []

    #  for each user
    for user in unique_users:
        user_recommendations = bfs_recommendations(graph, user)
        all_recommendations.append(user_recommendations)

    output_file_path = '/content/drive/MyDrive/daa dataset/comprise.csv'
    save_recommendations_to_csv(all_recommendations, output_file_path)

    print(f"Recommendations saved to: {output_file_path}")


DataFrame Head:
  User Name           Song Title
0      Judy                Sorry
1    Faythe  Rolling in the Deep
2     Sybil             Señorita
3      Ivan             Rockstar
4    Quincy          Blank Space

Unique Users:
['Judy' 'Faythe' 'Sybil' 'Ivan' 'Quincy' 'Ruth' 'Peggy' 'Grace' 'Eve'
 'Heidi' 'Leo' 'Trent' 'David' 'Alice' 'Mallory' 'Bob' 'Niaj' 'Kathy'
 'Olivia' 'Charlie']

Unique Songs:
['Sorry' 'Rolling in the Deep' 'Señorita' 'Rockstar' 'Blank Space'
 'Royals' 'Without Me' 'Love Me Like You Do' 'Stitches' 'Despacito'
 'Dance Monkey' 'Let Her Go' 'Someone Like You' 'Bad Guy' 'Hello'
 'All of Me' 'One Kiss' 'Shape of You' 'Sugar' 'Believer' 'Lose Yourself'
 'Cheap Thrills' 'Counting Stars' 'Perfect' 'Thinking Out Loud'
 'Blinding Lights' 'Havana' 'Closer' 'Faded' 'Memories']
Recommendations saved to: /content/drive/MyDrive/daa dataset/comprise.csv
